# Fit $k_{ij}$ and $r_c^{ABij}$ interactions parameter of Ethanol and CPME

---
Let's call $\underline{\xi}$ the optimization parameters of a mixture. In order to optimize them, you need to provide experimental phase equilibria data. This can include VLE, LLE and VLLE data. The objective function used for each equilibria type are shown below:


### Vapor-Liquid Equilibria Data


$$ OF_{VLE}(\underline{\xi}) = w_y \sum_{j=1}^{Np} \left[ \sum_{i=1}^c (y_{i,j}^{cal} - y_{i,j}^{exp})^2 \right] +  w_P \sum_{j=1}^{Np} \left[ \frac{P_{j}^{cal} - P_{j}^{exp}}{P_{j}^{exp}} \right]^2$$

Where, $Np$ is the number of experimental data points, $y_i$ is the vapor molar fraction of the component $i$ and $P$ is the bubble pressure. The superscripts $cal$ and $exp$ refers to the computed and experimental values, respectively. Finally, $w_y$ is the weight for the vapor composition error and $w_P$ is the weight for the bubble pressure error.


### Liquid-Liquid Equilibria Data
$$ OF_{LLE}(\underline{\xi}) = w_x \sum_{j=1}^{Np} \sum_{i=1}^c  \left[x_{i,j} - x_{i,j}^{exp}\right]^2 +  w_w \sum_{j=1}^{Np} \sum_{i=1}^c \left[ w_{i,j} - w_{i,j}^{exp} \right]^2 $$

Where, $Np$ is the number of experimental data points, $x_i$ and $w_i$ are the molar fraction of the component $i$ on the liquids phases. The superscripts $cal$ and $exp$ refers to the computed and experimental values, respectively. Finally, $w_x$ and $w_w$ are the weights for the liquid 1 ($x$) and liquid 2 ($w$) composition error.


### Vapor-Liquid-Liquid Equilibria Data
$$ OF_{VLLE}(\underline{\xi}) = w_x \sum_{j=1}^{Np}  \sum_{i=1}^c  \left[x_{i,j}^{cal} - x_{i,j}^{exp}\right]^2 + w_w \sum_{j=1}^{Np}  \sum_{i=1}^c  \left[w_{i,j}^{cal} - w_{i,j}^{exp}\right]^2 +  w_y \sum_{j=1}^{Np}  \sum_{i=1}^c  \left[y_{i,j}^{cal} - y_{i,j}^{exp}\right]^2 + w_P \sum_{j=1}^{Np}  \left[ \frac{P_{j}^{cal}}{P_{j}^{exp}} - 1\right]^2 $$

Where, $Np$ is the number of experimental data points, $y_i$, $x_i$ and $w_i$ are the molar fraction of the component $i$ on the vapor and liquids phases, respectively. The superscripts $cal$ and $exp$ refers to the computed and experimental values, respectively. Finally, $w_x$ and $w_w$ are the weights for the liquid 1 ($x$) and liquid 2 ($w$) composition error, $w_y$ is the weight for vapor composition error and $w_P$ is weight for three phase equilibria pressure error.

If there is data for more than one equilibria type, the errors can be added accordinly. So the objective funcion becomes:

$$ OF(\underline{\xi}) =OF_{ELV}(\underline{\xi}) + OF_{ELL}(\underline{\xi}) + OF_{ELLV}(\underline{\xi})$$

---

This notebook has te purpose of showing how to optimize the $k_{ij}$ and $r_c^{ABij}$ for a mixture with induced association. For these mixtures the interactions parameters  are shown below:

$$ \epsilon_{ij} = (1-k_{ij}) \frac{\sqrt{\sigma_i^3 \sigma_j^3}}{\sigma_{ij}^3} \sqrt{\epsilon_i \epsilon_j}  ;\quad\epsilon_{ij}^{AB} = \frac{\epsilon^{AB}  (self-associating)}{2} ;\quad  r^{ABij}_c (fitted)$$

First, it's needed to import the necessary modules

In [1]:
import numpy as np

from sgtpy import component, mixture, saftvrmie
from sgtpy.fit import fit_cross

Now that the functions are available it is necessary to create the mixture.

In [2]:
ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20)

cpme = component('cpme', ms =  2.32521144, sigma = 4.13606074, eps = 343.91193798, lambda_r = 14.15484877, 
                 lambda_a = 6.0, npol = 1.91990385,mupol = 1.27, sites =[0,0,1], cii = 3.5213681817448466e-19)

mix = mixture(ethanol, cpme)

Now the experimental equilibria data is read and a tuple is created. It includes the experimental liquid composition, vapor composition, equilibrium temperature and pressure. This is done with ```datavle = (Xexp, Yexp, Texp, Pexp)```

In [3]:
# Experimental data obtained from Mejia, Cartes, J. Chem. Eng. Data, vol. 64, no. 5, pp. 1970–1977, 2019

# Experimental temperature saturation in K
Texp = np.array([355.77, 346.42, 342.82, 340.41, 338.95, 337.78, 336.95, 336.29,
       335.72, 335.3 , 334.92, 334.61, 334.35, 334.09, 333.92, 333.79,
       333.72, 333.72, 333.81, 334.06, 334.58])

# Experimental pressure in Pa
Pexp = np.array([50000., 50000., 50000., 50000., 50000., 50000., 50000., 50000.,
       50000., 50000., 50000., 50000., 50000., 50000., 50000., 50000.,
       50000., 50000., 50000., 50000., 50000.])

# Experimental liquid composition
Xexp = np.array([[0.   , 0.065, 0.11 , 0.161, 0.203, 0.253, 0.301, 0.351, 0.402,
        0.446, 0.497, 0.541, 0.588, 0.643, 0.689, 0.743, 0.785, 0.837,
        0.893, 0.947, 1.   ],
       [1.   , 0.935, 0.89 , 0.839, 0.797, 0.747, 0.699, 0.649, 0.598,
        0.554, 0.503, 0.459, 0.412, 0.357, 0.311, 0.257, 0.215, 0.163,
        0.107, 0.053, 0.   ]])

# Experimental vapor composition
Yexp = np.array([[0.   , 0.302, 0.411, 0.48 , 0.527, 0.567, 0.592, 0.614, 0.642,
        0.657, 0.678, 0.694, 0.71 , 0.737, 0.753, 0.781, 0.801, 0.837,
        0.883, 0.929, 1.   ],
       [1.   , 0.698, 0.589, 0.52 , 0.473, 0.433, 0.408, 0.386, 0.358,
        0.343, 0.322, 0.306, 0.29 , 0.263, 0.247, 0.219, 0.199, 0.163,
        0.117, 0.071, 0.   ]])

datavle = (Xexp, Yexp, Texp, Pexp)

The function ```fit_cross``` optimize the $k_{ij}$ correction and $r_c^{ABij}$ distance. An initial guess is needed, as well as the mixture object, the index of the self-associating component and the equilibria data.  Optionally, the ```minimize_options``` option allows modifying the minimizer default parameters.

In [4]:
#initial guesses for kij and rcij
x0 = [0.01015194, 2.23153033]
fit_cross(x0, mix, assoc=0, datavle=datavle)

      fun: 9.863469849503824e-05
 hess_inv: array([[ 0.18584152,  2.61090939],
       [ 2.61090939, 41.36252561]])
      jac: array([-3.13878172e-06, -3.14077624e-07])
  message: 'Optimization terminated successfully.'
     nfev: 33
      nit: 8
     njev: 11
   status: 0
  success: True
        x: array([0.01147332, 2.23743327])

If the mixture exhibits other equilibria types you can supply this experimental data to the ``datalle`` or ``datavlle`` parameters.

- ``datalle``: (Xexp, Wexp, Texp, Pexp)
- ``datavlle``: (Xexp, Wexp, Yexp, Texp, Pexp)

You can specify the weights for each objetive function through the following parameters:

- ``weights_vle``: list or array_like, weights for the VLE objective function.
        - weights_vle[0] = weight for Y composition error, default to 1.
        - weights_vle[1] = weight for bubble pressure error, default to 1.
- ``weights_lle``: list or array_like, weights for the LLE objective function.
        - weights_lle[0] = weight for X (liquid 1) composition error, default to 1.
        - weights_lle[1] = weight for W (liquid 2) composition error, default to 1.
- ``weights_vlle``: list or array_like, weights for the VLLE objective function.
        - weights_vlle[0] = weight for X (liquid 1) composition error, default to 1.
        - weights_vlle[1] = weight for W (liquid 2) composition error, default to 1.
        - weights_vlle[2] = weight for Y (vapor) composition error, default to 1.
        - weights_vlle[3] = weight for equilibrium pressure error, default to 1.

Additionally, you can set options to the SciPy's ``minimize`` function using the ``minimize_options`` parameter.

For more information just run:
```fit_cross?```